# 9: Travel Time Ratio of 4.0

Model 9 is formulated the same way as Model 8, only with the TTR set to 4. (pp. 114)4)

Import larch and verify that the version number is correct (as of 1/30/2024 should be 6.0.17)

In [ ]:
import larch as lx

In [ ]:
lx.__version__

Import pandas, numpy

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = lx.examples.MTC(format="dataset")
d

In [ ]:
m = lx.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:idco data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:LinearFunction objects for each alternative code.

In [ ]:
from larch import PX, P, X

m.utility_co[2] = P("ASC_SR2") + P("hhinc#2,3") * X("hhinc")
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#2,3") * X("hhinc")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

In [ ]:
m.utility_ca = (
    +P("nonmotorized_time") * X("(altid>4) * tottime")
    + P("motorized_time") * (X("(altid <= 4) * ivtt") + 4 * X("(altid <= 4) * ovtt"))
    + PX("totcost")
)

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 9, TTR = 4.0"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 20, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(20)

Having created this model, we can then estimate it:

In [ ]:
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True)
m.calculate_parameter_covariance()
m.loglike()

In [ ]:
m.parameter_summary()

It is a little tough to read this report because the parameters show up in alphabetical order. We can use the reorder method to fix this and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        ".*cost.*",
        ".*time.*",
        ".*ivtt.*",
    ),
    (
        "Income",
        "hhinc.*",
    ),
    (
        "ASCs",
        "ASC.*",
    ),
)

In [ ]:
m.parameter_summary()

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
from pytest import approx

revealed_x = dict(zip(m.pnames, result.x))
expected_x = {
    "ASC_BIKE": -1.7736251954990991,
    "ASC_SR2": -2.3653672641251093,
    "ASC_SR3P": -3.801079706917033,
    "ASC_TRAN": -0.5312099316342354,
    "ASC_WALK": 0.42030620117952483,
    "hhinc#2,3": -0.0015594442515770833,
    "hhinc#4": -0.005540182661169404,
    "hhinc#5": -0.012343382441532672,
    "hhinc#6": -0.00934628232973145,
    "motorized_time": -0.017246556285183012,
    "nonmotorized_time": -0.06513081655018897,
    "totcost": -0.004836839766686574,
}
for k in expected_x:
    assert revealed_x[k] == approx(
        expected_x[k], 2e-2
    ), f"{k}, {revealed_x[k]/expected_x[k]}"